In [1]:
import sys
sys.path.append('../scripts/')
from s3_support import *

import dash, datetime
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

/home/ubuntu/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning:

urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!



In [21]:
yoy_vol_source = '''select
        source,
        sum(amount) as total_volume,
        sum(amount) / count(distinct form) as per_form_volume,
        sum(amount) / count(distinct org) as per_org_volume,
        count(distinct form) as form_count,
        count(distinct org) as org_count,
        extract(year from date) as year
    from transactions_tag
    where status='A' group by extract(year from date), source order by year'''
yoy = redshift_query_read(yoy_vol_source)

In [39]:
sources = yoy['source'].unique()

for source in sources:
    _df = yoy[yoy['source']==source].copy()
    
    _df.sort_values('year', ascending=True, inplace=True)
    for c in _df.columns:
        if c != 'source' and c != 'year':
            _df["{}_growth".format(c)] = _df[c].diff() / _df[c].shift(1)
    for c in _df.columns:
        if 'growth' in c:
            _df[c] = (_df[c] * 100.).apply(lambda x: "{:.2f}%".format(x))
        elif 'volume' in c:
            _df[c] = _df[c].apply(lambda x: "${:.2f}".format(x))

In [40]:
cols = ['year'] + [c for c in _df.columns if 'growth' in c]
_df[cols]

,year,total_volume_growth,per_form_volume_growth,per_org_volume_growth,form_count_growth,org_count_growth
63,2018,nan%,nan%,nan%,nan%,nan%
71,2019,87.35%,119.47%,136.35%,-14.63%,-20.73%


In [41]:
cols = [c for c in _df.columns if 'growth' not in c and c != 'source']
_df[cols]

,total_volume,per_form_volume,per_org_volume,form_count,org_count,year
63,$227168.54,$2770.35,$2770.35,82,82,2018
71,$425609.27,$6080.13,$6547.83,70,65,2019


In [42]:
_df.iloc[1:]

,source,total_volume,per_form_volume,per_org_volume,form_count,org_count,year,total_volume_growth,per_form_volume_growth,per_org_volume_growth,form_count_growth,org_count_growth
71,givi,$425609.27,$6080.13,$6547.83,70,65,2019,87.35%,119.47%,136.35%,-14.63%,-20.73%


In [43]:
len(_df)

2